# Large Language Models

In the ever-evolving landscape of artificial intelligence, large language models (LLMs) have emerged as transformative tools, reshaping the way we engage with and analyse language. These sophisticated models, honed on massive repositories of text data, possess the remarkable ability to comprehend, generate, and translate human language with unprecedented accuracy and fluency. Among the prominent LLM architectures, LangChain stands out for its efficiency and flexibility.

This notebook is designed to seamlessly run both locally and on Google Colab. For those who may only have a CPU, there are clear instructions on how to run the notebook without a GPU. Don't worry, simply follow the instructions for either GPU or CPU, depending on your setup.

Please note that using only a CPU will result in noticeably slower model performance.

---
## 1.&nbsp; Installations and Settings 🛠️

On Google Colab, you have access to free GPUs, whenever they're available. Let's utilise this advantage. To configure a Colab GPU, navigate to "Edit" and then "Notebook Settings". Select "GPU" and then click "Save".

To proceed, you'll need to install two libraries: Langchain and Llama.cpp. When operating this notebook locally, you only need to install these libraries once, and they'll remain on your computer. However, in Colab, they're not default libraries and must be installed for each session.

**LangChain** is a framework that simplifies the development of applications powered by large language models (LLMs)

**llama.cpp** enables us to execute quantised versions of models.

> Quantisation of LLMs is a process that reduces the precision of the numerical values in the model, such as converting 32-bit floating-point numbers to 8-bit integers. These models are therefore smaller and faster, allowing them to run on less powerful hardware with only a small loss in precision.

* If you're using a **CPU**, use the [standard installation](https://python.langchain.com/docs/integrations/llms/llamacpp#cpu-only-installation) of llama.cpp. Windows users might have to install [a couple of extra libraries too](https://python.langchain.com/docs/integrations/llms/llamacpp#installation-with-windows).
* If you have an **NVIDIA GPU**, you need to [activate cuBLAS](https://python.langchain.com/docs/integrations/llms/llamacpp#installation-with-openblas-cublas-clblast) with llama.cpp. cuBLAS is a library that speeds up operations on NVIDIA GPUs.
 * Some students using windows have also found [this guide](https://medium.com/@piyushbatra1999/installing-llama-cpp-python-with-nvidia-gpu-acceleration-on-windows-a-short-guide-0dfac475002d) useful.
* If you have a **silicon chip Apple with a GPU**, you need to [enable Metal](https://python.langchain.com/docs/integrations/llms/llamacpp#installation-with-metal).

In [ ]:
!pip3 install -qqq langchain --progress-bar off
# As this notebook is originally on Colab, here we'll use their NVIDIA GPU and activate cuBLAS
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install -qqq llama-cpp-python --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


Before we dive into the examples, let's download the large language model (LLM) we'll be using. For these exercises, we've selected a [quantised version of Mistral AI's Mistral 7B model](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF). While this is a great choice, it's by no means the only option. We encourage you to explore and try different models to discover the unique strengths and weaknesses of each. Even models of similar size can exhibit surprisingly different capabilities.

> Since we're working in Colab, we'll need to download the LLM for each session.
<br>
If you're working locally, you can download the model once. The model is then on your computer and doesn't need to be downloaded each time. Change the `--local-dir` to your folder of choice.

In [ ]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpsj5g8lkp
mistral-7b-instruct-v0.1.Q4_K_M.gguf: 100% 4.37G/4.37G [00:59<00:00, 73.1MB/s]
./mistral-7b-instruct-v0.1.Q4_K_M.gguf


---
## 2.&nbsp; Setting up your LLM 🧠

Langchain simplifies LLM deployment with its streamlined setup process. A single line of code configures your LLM, allowing you to tailor the parameters to your specific needs.

If you want to know more about Llama.cpp, you can [read the docs here](https://llama-cpp-python.readthedocs.io/en/latest/api-reference/). Alternatively, here are the [LangChain docs for Llama.cpp](https://python.langchain.com/docs/integrations/llms/llamacpp).

Here's a brief overview of some of the parameters:
* **model_path:** The path to the Llama model file that will be used for generating text.
* **max_tokens:** The maximum number of tokens that the model should generate in its response.
* **temperature:** A value between 0 and 1 that controls the randomness of the model's generation. A lower temperature results in more predictable, constrained output, while a higher temperature yields more creative and diverse text.
* **top_p:** A value between 0 and 1 that controls the diversity of the model's predictions. A higher top_p value prioritizes the most probable tokens, while a lower top_p value encourages the model to explore a wider range of possibilities.
* **n_gpu_layers:** The default setting of 0 will cause all layers to be executed on the CPU. Setting n_gpu_layers to 1 will cause the first layer of the model to be executed on the GPU, while the remaining layers are executed on the CPU. Setting n_gpu_layers to 2 will cause the first two layers of the model to be executed on the GPU, while the remaining layers are executed on the CPU, and so on. -1 will cause all layers to be offloaded to the GPU. In general, it is a good idea to experiment with different values of n_gpu_layers to find the best balance between performance and memory usage for your specific application.

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path = "/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens = 2000,
               temperature = 0.1,
               top_p = 1,
               n_gpu_layers = -1)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

If you're using a GPU, check the output of this cell ☝️
  * If you're using cuBLAS, you'll see `BLAS = 1` if it's installed correctly.
  * If you're using Metal, you'll see `NEON = 1` if it's installed correctly.

---
## 3.&nbsp; Asking your LLM questions 🤖
Play around and note how small changes make a big difference.

In [ ]:
answer_1 = llm.invoke("Which animals live at the north pole?")
print(answer_1)


llama_print_timings:        load time =    5907.86 ms
llama_print_timings:      sample time =      20.56 ms /    36 runs   (    0.57 ms per token,  1751.14 tokens per second)
llama_print_timings: prompt eval time =    5907.78 ms /     8 tokens (  738.47 ms per token,     1.35 tokens per second)
llama_print_timings:        eval time =   21417.20 ms /    36 runs   (  594.92 ms per token,     1.68 tokens per second)
llama_print_timings:       total time =   27475.34 ms /    44 tokens



Polar bears, arctic foxes, walruses, seals, and narwhals are some of the animals that live at the North Pole.


In [ ]:
answer_2 = llm.invoke("Write a poem about animals that live at the north pole.")
print(answer_2)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5907.86 ms
llama_print_timings:      sample time =     208.20 ms /   336 runs   (    0.62 ms per token,  1613.81 tokens per second)
llama_print_timings: prompt eval time =    5914.04 ms /    12 tokens (  492.84 ms per token,     2.03 tokens per second)
llama_print_timings:        eval time =  205891.43 ms /   335 runs   (  614.60 ms per token,     1.63 tokens per second)
llama_print_timings:       total time =  213350.67 ms /   347 tokens




At the North Pole, where the ice is thick and cold,
Lives a world of animals brave and bold.
From the mighty polar bear to the tiny snowy owl,
Each one plays an important role in this arctic dwelling.

The polar bear rules the land with strength and might,
A creature of power, a true arctic sight.
With its thick fur coat and sharp claws for fighting,
It's a force to be reckoned with, always hunting.

But there are others who live in this world of snow,
Each one playing its part, never letting go.
The arctic fox, with its white fur and cunning ways,
Is always on the hunt for its next meal today.

The snowy owl, with its beautiful white plumage,
Soars high above the world, a true arctic sage.
With its keen eyesight, it sees all from above,
A guardian of the north, a creature of love.

And let's not forget the seals, who live beneath the ice,
Basking in the warmth of this arctic paradise.
They flip and flop, playing games and having fun,
In this world of animals, where everyone's happy 

In [ ]:
answer_3 = llm.invoke("Explain the central limit theorem like I'm 5 years old.")
print(answer_3)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5907.86 ms
llama_print_timings:      sample time =      56.70 ms /    92 runs   (    0.62 ms per token,  1622.49 tokens per second)
llama_print_timings: prompt eval time =    6490.39 ms /    15 tokens (  432.69 ms per token,     2.31 tokens per second)
llama_print_timings:        eval time =   54930.91 ms /    91 runs   (  603.64 ms per token,     1.66 tokens per second)
llama_print_timings:       total time =   61809.26 ms /   106 tokens




The central limit theorem is a math rule that says if you take lots of numbers and add them up, it doesn't really matter where the numbers came from or how many there are, the result will still be pretty close to normal. Like if you roll a dice 100 times, the sum of all the rolls will probably be around the middle number (50), even though some rolls might be higher or lower than that.


The answers provided by the 7B model may not seem as impressive as those from the latest OpenAI or Google models, but consider the significant size difference - they perform very well. These models may not have the most extensive knowledge base, but for our purposes, we only need them to generate coherent English. We'll then infuse them with specialised knowledge on a topic of your choice, resulting in a local, specialised model that can function offline.

---
## 4.&nbsp; Challenge 😀
Play around with this, and other, LLMs. keep a record of your findings:
1. Pose different questions to the model, each subtly different from the last. Observe the resulting outputs. Smaller models tend to be highly sensitive to minor changes in language and grammar.
2. Experiment with the parameters, one at a time, to assess their impact on the output.
3. Attempt to load different models: Explore the [models page on HuggingFace](https://huggingface.co/models). You can use the left hand menu to find `Text Generation` under `Natural Language Processing`. Then use the filter bar for `GGUF` to find already quantised models.

You can alter the download command accordingly. In this note book we used the command:

In [ ]:
# !huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

This downloads the version `mistral-7b-instruct-v0.1.Q4_K_M.gguf` of the model `TheBloke/Mistral-7B-Instruct-v0.1-GGUF` from huggingface. You can read about the different versions on the models `model card`.

To adapt this just change the model and the version to your new choice.

`!huggingface-cli download {model_name} {model_version} --local-dir . --local-dir-use-symlinks False`

For example:

In [ ]:
# !huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

The code above would download a [quantised version of Meta's Llama 2 7B chat](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/tree/main).








